### Exercise: Gauss-Seidel Iteration

This exercise focuses on solving the Poisson equation in one dimension with homogeneous Dirichlet conditions:
$$
- u''(x) = b(x), \quad u(0) = u(1) = 0, \quad b(x) := \exp(x).
$$
A discretization of this equation on a uniform grid using the finite difference method leads to the following linear system:
$$
\frac{1}{h^2}
\begin{pmatrix}
    2 & -1 \\
    -1 & 2  & -1 \\
       & -1 & 2      & -1 \\
       &    & \ddots & \ddots & \ddots & \\
       &    &        & -1    & 2      & -1 \\
       &    &        &     & -1      & 2 \\
\end{pmatrix}
\begin{pmatrix}
    u_1 \\
    u_2 \\
    u_3 \\
    \vdots \\
    u_{n-2} \\
    u_{n-1}
\end{pmatrix}
=
\begin{pmatrix}
    b(x_1) \\
    b(x_2) \\
    b(x_3) \\
    \vdots \\
    b(x_{n-2}) \\
    b(x_{n-1})
\end{pmatrix},
\quad
h := \frac{1}{n},
\quad
x_i := ih.
$$
where $h$ is the spacing between the discretization points, and $(u_1, u_2, \cdots, u_{n-1})$ are the sought values of the unknown function $u$ at the interior points of the domain $[0, 1]$.

1. Compute the solution of the system for $n = 50$ using Julia's `\` method.

   > **Hint**: To build the matrix of the linear system, one can use `LinearAlgebra.SymTridiagonal` and the `fill` function.

2. Implement the Gauss-Seidel method (or its generalization, the relaxation method) to solve the linear system for $n = 50$, this time without using Julia's `\` and `inv` functions or any software library. Initialize the iteration with $\mathbf x_0 = \mathbf 0$.

3. Verify on a graph that the obtained approximate solution is close to the exact solution, which is given by $$u(x) = \exp(x) - 1 - (\exp(1) - 1)x.$$

### Exercise: Conjugate Gradient method

Consider a linear system of the form
$$
\tag{*} \mathsf A \mathbf x = \mathbf b,
$$
where $\mathsf A \in \mathbb R^{n\times n}$ is a positive definite symmetric matrix. The goal of this exercise is to theoretically derive and then implement the conjugate gradient method. Let $\mathbf x_*$ be the exact solution of the system.

1. **Theoretical Derivation of the Method**.
   Assuming a vector $\mathbf x_0$ is given, define, for $k \in \mathbb N$,
   the Krylov space
   $$
   \mathcal K_k := \text{Span} \Bigl\{ \mathbf r_0, \mathsf A \mathbf r_0, \mathsf A^2 \mathbf r_0, \dotsc, \mathsf A^{k-1} \mathbf r_0 \Bigr\},
   \qquad \mathbf r_0 := \mathsf A \mathbf x_0 - \mathbf b.
   $$
   Let $\mathbf x_k$ be the minimizer of $f$ in the affine space $\mathbf x_0 + \mathcal K_k$,
   where $f$ is the function
   $$
   f(\mathbf x) := \frac{1}{2} \mathbf x^T \mathsf A \mathbf x - \mathbf b^T \mathbf x.
   $$
   The goal of this exercise is to arrive at a simple recurrence formula to construct $\mathbf x_k$ iteratively.

   - Show that there exists $k_* \in \{0, \dotsc, n\}$ such that $\mathbf x_k = \mathbf x_*$ for all $k \geq k_*$.

     *Hint 1*: Note that $\mathsf A \mathbf x = \mathbf b$ if and only if $\mathsf A (\mathbf x - \mathbf x_0) = - \mathbf r_0$.

     *Hint 2*: Show that there exists $k_* \in \{0, \dotsc, n\}$ such that $\mathsf A \mathcal K_{k_*} = \mathcal K_{k_*}$,
     and in particular, $\mathbf r_0 \in \mathsf A \mathcal K_{k_*}$.

   - Show that
     $f(\mathbf x) = \frac{1}{2} \lVert \mathbf x - \mathbf x_* \rVert_{\mathsf A}^2 + f(\mathbf x_*)$,
     where $\langle \mathbf x, \mathbf y \rangle_{\mathsf A} := \mathbf x^T \mathsf A \mathbf y$
     and $\lVert \cdot \rVert_{\mathsf A}$ is the associated norm.

     *Note*: Notice that this point implies that $\mathbf x_k - \mathbf x_0$ is the $\mathsf A$-orthogonal projection of $\mathbf x_* - \mathbf x_0$ onto $\mathcal K_k$.

   - Let $\mathbf r_k := \mathsf A \mathbf x_k - \mathbf b$.
     Show that $\mathbf r_k = \mathbf r_0 + \mathsf A (\mathbf x_k - \mathbf x_0)$,
     and deduce that $\mathbf r_k \in \mathcal K_{k+1}$.

   - Show that $\langle \mathbf r_{k}, \mathbf v \rangle = 0$ for all $\mathbf v \in \mathcal K_{k}$.

     *Hint*: Note that $\mathbf r_k = \nabla f(\mathbf x_k)$.

   - Deduce from the two previous points that if $\mathbf r_k \neq \mathbf 0$,
     then $\mathbf r_k \in \mathcal K_{k+1} \setminus \mathcal K_k$.

   - Also deduce that $\langle \mathbf r_{k}, \mathbf v \rangle_{\mathsf A} = 0$ for all $\mathbf v \in \mathcal K_{k-1}$,
     for $k \geq 1$.

     *Hint*: Use the definition of the Krylov spaces $\mathcal K_k$.

   - Conclude that if $\mathbf r_k \neq \mathbf 0$,
     then the vector $\mathbf d_k$ defined by the recurrence relation
     $$
     \tag{CG1}
     \mathbf d_{k} = \mathbf r_{k} - \frac{\mathbf r_{k}^T \mathsf A \mathbf d_{k-1}}{\mathbf d_{k-1}^T \mathsf A \mathbf d_{k-1}} \mathbf d_{k-1},
     \qquad \mathbf d_0 = \mathbf r_0
     $$
     <span id="CG1"></span>
     is the unique (up to a multiplicative constant) element of $\mathcal K_{k+1}$
     that is $\mathsf A$-orthogonal to $\mathcal K_k$.

   - Show that $\mathbf x_{k+1} - \mathbf x_{k}$ is $\mathsf A$-orthogonal to $\mathcal K_k$,
     hence $\mathbf x_{k+1} = \mathbf x_{k} - \omega_k \mathbf d_k$ for some $\omega_k \in \mathbb R$ when $\mathbf r_k \neq 0$.

     *Hint*: Note that $\langle \mathbf x_{k+1} - \mathbf x_k, \mathbf v \rangle_{\mathsf A} = \langle \mathbf r_{k+1} - \mathbf r_k, \mathbf v \rangle$,
     and use one of the results proven above.

   - Finally, show that
     $$
     \tag{CG2}
     \mathbf x_{k+1} = \mathbf x_k - \frac{\mathbf r_k^T \mathbf d_k}{\mathbf d_k^T \mathsf A \mathbf d_k} \mathbf d_k.
     $$
     <span id="CG1"></span>
     *Hint*: Find the parameter $\omega_k$ from the previous point by minimizing $f(\mathbf x_k - \omega_k \mathbf d_k)$.

1. **Implementation of the Method**.
   The relations <a href="#CG1">(CG1)</a> and <a href="#CG2">(CG2)</a> allow for constructing vectors $(\mathbf x_k)$ recursively,
   with a cost per iteration independent of $k$; this is the **conjugate gradient method**.

   - Based on these equations,
     implement a Julia function to solve the system <a href="#system">(*)</a>.

   - Test your implementation on the matrices $\mathsf A$ and $\mathbf b$ in the code below,
     which correspond to a finite difference discretization of the Poisson equation on the domain $\Omega = (0, 2) \times (0, 1)$,
     with homogeneous Dirichlet boundary condition:
     $$
     \begin{aligned}
         - \Delta u(x, y) &= b(x, y), \qquad (x, y) \in \Omega, \\
         u(x, y) &= 0, \quad \qquad \quad (x, y) \in \partial \Omega.
     \end{aligned}
     $$
     The right-hand side considered is
     $$
        b(x, y) = \sin(4\pi x) + \sin(2\pi y).
     $$
     Use a vector of 1 as the initial vector $\mathbf x_0$ which can be built using the `ones` function.

   - Create a graph illustrating the evolution of the residual $\lVert \mathbf r_k \rVert$, the error $\lVert \mathbf x_k - \mathbf x_* \rVert$,
     and the error $\lVert \mathbf x_k - \mathbf x_* \rVert_{\mathsf A}$ as a function of $k$.
     Choose appropriate scales for the graph (linear-linear, log-log, linear-log, or log-linear).

In [ ]:
using LaTeXStrings
using LinearAlgebra
using Plots
import SparseArrays

# Domain size
Lx, Ly = 2, 1

# Number of discretization points along x and y, including the boundary points
nx, ny = 101, 101

function discretize(nx, ny)
    hx, hy = Lx/(nx - 1), Ly/(ny - 1)
    Dxx = (1/hx^2) * SymTridiagonal(2ones(nx-2), -ones(nx-3))
    Dyy = (1/hy^2) * SymTridiagonal(2ones(ny-2), -ones(ny-3))
    A = kron(Dxx, I(ny-2)) + kron(I(nx-2), Dyy)
    xgrid = Lx/(nx-1) * (1:nx-2)
    ygrid = Ly/(ny-1) * (1:ny-2)
    x_2d = reshape([x for y in ygrid, x in xgrid], (nx-2)*(ny-2))
    y_2d = reshape([y for y in ygrid, x in xgrid], (nx-2)*(ny-2))
    b = sin.(4π*x_2d) + sin.(2π*y_2d)
    return SparseArrays.SparseMatrixCSC(A), b
end

function plot_solution(f)
    f = reshape(f, ny-2, nx-2)
    z = [zeros(nx)'; zeros(ny-2) f zeros(ny-2); zeros(nx)']  # Add boundary
    xgrid = Lx/(nx-1) * (0:nx-1)
    ygrid = Ly/(ny-1) * (0:ny-1)
    heatmap(xgrid, ygrid, z, c=:viridis, levels=50)
end

In [ ]:
# Your function should return a list of iterates x_0, x_1, …
function conjugate_gradients(A, b)
    # Relpace the line below with your code
    [A\b]
end

In [ ]:
A, b = discretize(nx, ny)
xs = conjugate_gradients(A, b)
plot_solution(xs[end])

In [ ]:
# Produce plots of the residual and errors here